### What agriculture water use data to use for a demand site?

old
### Use Case 2.2: What differences are there across datasets in agriculture water use in an area?

This notebook demonstrates basic WaMDaM use cases analysis using scientific Python libraries such as [pandas](https://pandas.pydata.org/) and [plotly](https://plot.ly/).  It reads WaMDaM SQLite data from a published HydroShare Generic Resource, runs SQL script, and them uses Python plotly to visualize the results

This use case identifies five volume-elevation curves for Hyrum Reservoir, Utah from three datasets: USBOR, Utah Dams, and WEAP model datasets

For more info: http://docs.wamdam.org/UseCases/use_case_2/#use-case-2.2



In [1]:
import os
from utilities import hydroshare
#from hs_restclient import HydroShare

import sqlite3
import numpy as np
import pandas as pd

In [ ]:
!pip install plotly  # python 2.7
!pip3 install plotly # python3
!conda install --yes -c plotly plotly -y


WaMDaM database test file (SQLite) on HydroShare
https://www.hydroshare.org/resource/1601e9f029984a87affcd94af6b4bad0/



import os
from utilities import hydroshare
#from hs_restclient import HydroShare

import sqlite3
import numpy as np
import pandas as pd

Next we need to establish a secure connection with HydroShare. This is done by simply instantiating the hydroshare class that is defined within hs_utils. In addition to connecting with HydroShare, this command also sets environment variables for several parameters that may useful to you:

Your username
The ID of the resource which launched the notebook
The type of resource that launched this notebook
The url for the notebook server.

In [2]:
# establish a secure connection to HydroShare
hs = hydroshare.hydroshare()

Adding the following system variables:
   HS_USR_NAME = amabdallah
   HS_RES_ID = e3fc4c100a1440be8ed2669e3803cef9
   HS_RES_TYPE = genericresource
   JUPYTER_HUB_IP = jupyter.cuahsi.org

These can be accessed using the following command: 
   os.environ[key]

   (e.g.)
   os.environ["HS_USR_NAME"]  => amabdallah
Successfully established a connection with HydroShare


In [3]:
### Retrieve a resource using its ID

# The data for our processing routines can be retrieved using the `getResourceFromHydroShare` function by passing it the global identifier from the url above
# get some resource content. The resource content is returned as a dictionary
# Abdallah, A. (2018). Bear River Datasets, HydroShare, http://www.hydroshare.org/resource/bec9b20118804d119c4bfc232caea559
content = hs.getResourceFromHydroShare('bec9b20118804d119c4bfc232caea559')



This resource already exists in your userspace.
Would you like to overwrite this data [Y/n]? y
Download Finished                               
Successfully downloaded resource bec9b20118804d119c4bfc232caea559


In [4]:
# Use SQLite engine to connect to the SQLite file. 
# Then we can run queries against it within this notebook :)  
conn = sqlite3.connect(hs.content["BearRiverDatasets_Jan2018.sqlite"])
print 'connected'

connected


In [ ]:
# Query the water use data

# 4.2MultiAttributeValues.csv
import urllib

# for simplicity, we call the query directly from GitHub as a text. You may use the query here but it will be too long to show in a cell 
#https://github.com/WamdamProject/WaMDaM_UseCases/blob/master/UseCases_files/4Queries_SQL/UseCase2/UseCase2.3/4_MultiAttributeValues.sql

txt1 = urllib.urlopen("https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/4Queries_SQL/UseCase2/UseCase2.3/3_MultipleTimeSeriesColumnsSameTimeStamp.sql").read()

# pass the query to the SQLite connection
df = pd.read_sql_query(txt1, conn)

# Save the query result into a CSV file
df.to_csv('query_resut.csv')

# to print the table result here within the notebook, just uncomment the df line below
#df


In [ ]:
# Import plotly libraries and set it to the notebook mode to embed the figures within a cell
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode()

In [ ]:
# UseCase2.2_dentifyDemandSites_TimeSeriesValues.py

# plot time series data aggregated in space and time from multiple sources

# Jan 25, 2018
# Adel Abdallah

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from random import randint
import pandas as pd

#6.3dentifyDemandSites_TimeSeriesValues.csv

## read the input data from GitHub csv file which is a direct query output
#To get the data block (WaterYear,CumulativeAnnual) for each curve, you need to look up two columns:
#ScenarioName and then AttributeName. So the combination of these two columns will have their separate set of data.
df = pd.read_csv('https://raw.githubusercontent.com/WamdamProject/WaMDaM_UseCases/master/UseCases_files/5Results_CSV/3.3dentifyDemandSites_TimeSeriesValues.csv')

subsets = df.groupby('AttributeName')

data = []


# for each subset (curve), set up its legend and line info manually so they can be edited
subsets_settings = {
    '11 sites (time series): WEAP Model 2017': {
        'dash': 'solid',
        'legend_index': 0,
        'legend_name': '<br> 11 sites (seasonal): WEAP Model 2017 <br> "Monthly Demand" ',
        'width':'3',
        'color':'rgb(41, 10, 216)'
        },      
    'Diversions /surface water': {
        'dash': 'solid',
        'legend_index': 1,
        'legend_name': '<br> 1 site (time series): WaDE <br> "Diversions/surface water"',
        'width':'3',
        'color':'rgb(38, 77, 255)'        
        },        
        
    'Water Use /surface and ground': {
        'dash': 'dash',
        'legend_index': 2,
        'legend_name': '<br> 1 site (time series): WaDE  <br> "Water Use/surface and ground water"',
        'width':'3',
        'color':'rgb(63, 160, 255)'
        },
    
    'dReq': { # this one is the name of subset as it appears in the csv file
        'dash': 'solid',     # this is properity of the line (curve)
        'legend_index': 3,   # to order the legend
        'legend_name': '<br> 5 sites (time series): WASH Model <br> "dReq"',  # this is the manual curve name 
         'width':'3',
        'color':'rgb(114, 217, 255)'
        },
    'Monthly Demand': {
        'dash': 'solid',
        'legend_index': 4,
        'legend_name': '<br> 1 site (time series): WEAP Model 2010 <br> "Monthly Demand"',
        'width':'3',
        'color':'rgb(170, 247, 255)'
        },

            }
    

# This dict is used to map legend_name to original subset name
subsets_names = {y['legend_name']: x for x,y in subsets_settings.iteritems()}


for subset in subsets.groups.keys():
    print subset
    scenario_name_data = subsets.get_group(name=subset)
    subsets_of_scenario = scenario_name_data.groupby("AttributeName")
    for group in subsets_of_scenario.groups.keys():
        s = go.Scatter(
                x = subsets_of_scenario.get_group(name=group).WaterYear,
                y = subsets_of_scenario.get_group(name=group).CumulativeAnnual,
                name = subsets_settings[subset]['legend_name'],
                line = dict(
                        color =subsets_settings[subset]['color'],
                        width =subsets_settings[subset]['width'], 
                        dash=subsets_settings[subset]['dash']
                        ),
                mode = 'lines',
                opacity = 1)
        data.append(s)
        
      
        
# horizontal line
horizontal_line = go.Scatter(
    x=[2005, 2016],
    y=[232642.28, 232642.28],
    mode='lines',
    name = '<br> 11 sites (seasonal): WEAP Model 2017 <br> "Monthly Demand" ',
    hoverinfo='11 sites: WEAP Model 2017',
    showlegend=True,
    line=dict(
        shape='hv',
        color = 'rgb(38, 15, 153)',
        width='3'
    )
)
data.append(horizontal_line)


# Legend is ordered based on data, so we are sorting the data based 
# on desired legend order indicarted by the index value entered above
data.sort(key=lambda x: subsets_settings[subsets_names[x['name']]]['legend_index'])        
   
layout = dict(
    #title = "Use Case 6",
    yaxis = dict(
        title = "Total volume per water year <br> (acre-feet)",       
        tickformat= ',',
        showline=True,
        range = ['0', '260000'],

                ),
    xaxis = dict(
        range = ['2003', '2016'],
        ticks='outside',
        tickwidth=0.5,
        ticklen=25,
        showline=True
                ),
    legend=dict(x=0.86,y=0.445,
                bordercolor='#00000',
                borderwidth=2  
               ),
     width=1650,
    height=1000,
    margin=go.Margin(
        l=250,
        b=100       ),
    #paper_bgcolor='rgb(233,233,233)',
    #plot_bgcolor='rgb(233,233,233)',
    font=dict(size=32)

)


# see the label Lines with Annotations
# https://plot.ly/python/line-charts/      
annotations = []

label = ['11 sites', '1 site', '1 site', '5 sites','1 site']

for legend_index in subsets_settings:
    annotations.append(dict(xref='paper', x=2004, y=subsets_settings[legend_index],
                                  xanchor='right', yanchor='middle',
                                  text=label)
                      )

layout['annotations'] = annotations
                

# create a figure object          
fig = dict(data=data, layout=layout)

#py.iplot(fig, filename = "UseCase2.2_dentifyDemandSites_TimeSeriesValues") 

## it can be run from the local machine on Pycharm like this like below
## It would also work here offline but in a seperate window  
plotly.offline.plot(fig, filename = "UseCase2.2_dentifyDemandSites_TimeSeriesValues")       

offline.iplot(fig,filename = 'UseCase2.2_dentifyDemandSites_TimeSeriesValues',
             image='png')
# it might take 30-60 seconds to load the html interactive image 

# Connect to the WEAP API
First make sure to have a copy of the Water Evaluation And Planning" system (WEAP) installed on your local machine (Windows). 
You will need to have an active licence to use the API
For more info, see here http://www.weap21.org/index.asp?action=40

## WEAP API info 
http://www.weap21.org/WebHelp/API.htm

## Install dependency and register WEAP
### A. Install pywin32 extensions which provide access to many of the Windows APIs from Python.
**Choose on option**
1. Install using an executable basedon your python version. I used Python 2.7
https://github.com/mhammond/pywin32/releases

2. Install from source code (for advanced users) 
https://github.com/mhammond/pywin32

### B. Register WEAP with Windows 
Use Windows "Command Prompt" as Administrator, go to WEAP install directory (e.g. `cd C:\Program Files (x86)\WEAP`) and simply run the following command: 

`WEAP /regserver`

In [8]:
python Scripts/pywin32_postinstall.py -install

SyntaxError: invalid syntax (<ipython-input-8-6527b68e6ed0>, line 1)

## Connect to the WEAP model

In [7]:
# this library is needed to connect to the WEAP API
import win32com.client

# this command will open the WEAP software (if closed) and get the last active model
# you could change the active area to another one inside WEAP or by passing it to the command here
#WEAP.ActiveArea = "BearRiverFeb2017_V10.9"

WEAP=win32com.client.Dispatch("WEAP.WEAPApplication")

# get the active WEAP Area (model) to serve data into it 
ActiveArea=WEAP.ActiveArea.Name 
print  'ActiveArea= '+ActiveArea

# get the active WEAP scenario to serve data into it 

ActiveScenario= WEAP.ActiveScenario.Name
print 'ActiveScenario= '+ActiveScenario




ImportError: No module named win32com.client

<a id='section4'></a>
### 4. Creating a new HydroShare resource

The best way to save your data is to put it back into HydroShare and is done using the `createHydroShareResource` function. The first step is to identify the files you want to save to a HydroShare.  The cell below lists all the files in the current working directory.

In [ ]:
# Grab all the files in the folder where you are working  
files = !ls
print('My files: %s' % files)

In [ ]:
# lets save this content as a new resource in HydroShare
abstract = 'This is a demo of the HydroShare Python Notebook Server for WaMDaM'
title = 'WaMDaM_Use_Case2.2'    
keywords = ['Demo', 'JupyterHub']  
rtype = 'genericresource'          

# create a list of files that will be added to the HydroShare resource.
#files = ['WaMDaM_use_cases_Multi_columns.ipynb.ipynb']  # this notebook
        

In [ ]:
# create a list of files that will be added to the HydroShare resource.
    
files = [hs.content['BearRiverDatasets_Jan2018.sqlite'],'WaMDaM_Use_Case2.2.ipynb']  # this notebook

In [ ]:
    
# create the new resource
resource_id = hs.createHydroShareResource(abstract, 
                                          title, 
                                          keywords=keywords, 
                                          resource_type=rtype, 
                                          content_files=files, 
                                          public=True)